# Metaheuristics similarity analysis using `MetaheuristicsSimilarityAnalyzer`

This example is a tutorial and a showcase of the MSA framework capabilities and features for similarity analysis of metaheuristics.

### Import tools and libraries

In [ ]:
# Analysis tools
from msa.tools.metaheuristics_similarity_analyzer import MetaheuristicsSimilarityAnalyzer
from msa.tools.meta_ga import MetaGAFitnessFunction, MetaGA

# Diversity metrics
from msa.diversity_metrics.individual_diversity.idt import IDT
from msa.diversity_metrics.individual_diversity.ifiqr import IFIQR
from msa.diversity_metrics.individual_diversity.ifm import IFM
from msa.diversity_metrics.individual_diversity.isi import ISI
from msa.diversity_metrics.population_diversity.dpc import DPC
from msa.diversity_metrics.population_diversity.fdc import FDC
from msa.diversity_metrics.population_diversity.pfm import PFM
from msa.diversity_metrics.population_diversity.pfsd import PFSD

# Optimization problem from the NiaPy framework
from niapy.problems.schwefel import Schwefel

# Other tools
import os
import numpy as np
import matplotlib
matplotlib.use("inline")

# Define constants for later use
base_archive_path = "archive"
msa_pkl_filename = "msa_example"

### Define gene spaces used for the analysis
Name of the algorithm can be a string corresponding with the name of the algorithm class from the `niapy.algorithms` framework, or a class inheriting from the niapy `Algorithm` class. Names of the hyperparameters must correspond with the names of selected algorithm constructor arguments.

---
**_TIP:_** In the following step are some gene space examples to use in the following code. Try defining gene spaces for other algorithms from niapy framework.

---
**_NOTE:_** Make sure that algorithm returns `sorted_idx` as the result of `argsort` under `Additional arguments` in `run_iteration` method if it is using sorting. Some algorithms will require adaptation for the analysis to work as intended. See `msa.algorithms.fa` as an example.

---

In [ ]:
from msa.algorithms.pso import ParticleSwarmAlgorithm # Modified using `limit` repair
from msa.algorithms.fa import FireflyAlgorithm # Modified for faster processing with help of @jit and use of `sorted_idx`
from niapy.algorithms.basic.de import DifferentialEvolution

PSO_gene_space = {
    ParticleSwarmAlgorithm: {
        "c1": {"low": 0.01, "high": 2.5, "step": 0.01},
        "c2": {"low": 0.01, "high": 2.5, "step": 0.01},
        "w": {"low": 0.0, "high": 1.0, "step": 0.01},
    }
}

BA_gene_space = {
    "BatAlgorithm": {
        "loudness": {"low": 0.01, "high": 1.0, "step": 0.01},
        "pulse_rate": {"low": 0.01, "high": 1.0, "step": 0.01},
        "alpha": {"low": 0.9, "high": 1.0, "step": 0.001},
        "gamma": {"low": 0.0, "high": 1.0, "step": 0.01},
    }
}

FA_gene_space = {
    FireflyAlgorithm: {
        "alpha": {"low": 0.01, "high": 1.0, "step": 0.01},
        "beta0": {"low": 0.01, "high": 1.0, "step": 0.01},
        "gamma": {"low": 0.0, "high": 1.0, "step": 0.001},
        "theta": {"low": 0.95, "high": 1.0, "step": 0.001},
    }
}

DE_gene_space = {
    DifferentialEvolution: {
        "differential_weight": {"low": 0.01, "high": 1.0, "step": 0.01},
        "crossover_probability": {"low": 0.01, "high": 1.0, "step": 0.01},
    }
}


### Choose the optimization problem and the diversity metrics to use for the analysis
* Individual diversity metrics are available in `msa.diversity_metrics.individual_diversity`.
* Population diversity metrics are available in `msa.diversity_metrics.population_diversity`.

---
**_TIP:_** To add custom diversity metrics implement a class inheriting from either `PopDiversityMetric` or `IndivDiversityMetric` available in `msa.tools.optimization_data`.

---

**_NOTE:_** Make sure to define metrics with the optimization problem used for the analysis, since some of them use its parameters.

---

In [ ]:
OPTIMIZATION_PROBLEM = Schwefel(dimension=20)

POP_DIVERSITY_METRICS = [
    DPC(OPTIMIZATION_PROBLEM),
    FDC(OPTIMIZATION_PROBLEM, [420.968746], True),
    PFSD(),
    PFM(),
]
INDIV_DIVERSITY_METRICS = [
    IDT(),
    ISI(),
    IFM(),
    IFIQR(),
]

### Define a MetaGA instance used for the analysis
* Set `fitness_function_type` to `MetaGAFitnessFunction.TARGET_PERFORMANCE_SIMILARITY`.
* Arguments starting with `ga_` correspond with the arguments of the class `GA` from `pygad` module.
* Set `gene_space` to gene space of the algorithm to be optimized.

---
**_TIP:_** Try out gene spaces defined in previous step or add new ones to analyze similarity of other algorithms.

---

**_NOTE:_** MetaGA in this configuration can be used as a standalone tool, however to get the most out of the analysis it is recommended to use it in combination with `MetaheuristicsSimilarityAnalyzer` as shown in the next step.

---

In [ ]:
meta_ga = MetaGA(
    fitness_function_type=MetaGAFitnessFunction.TARGET_PERFORMANCE_SIMILARITY,
    ga_generations=3,
    ga_solutions_per_pop=5,
    ga_percent_parents_mating=60,
    ga_parent_selection_type="tournament",
    ga_k_tournament=2,
    ga_crossover_type="uniform",
    ga_mutation_type="random",
    ga_crossover_probability=0.9,
    ga_mutation_num_genes=1,
    ga_keep_elitism=1,
    gene_space=FA_gene_space,
    pop_size=30,
    max_evals=10000,
    num_runs=30,
    problem=OPTIMIZATION_PROBLEM,
    pop_diversity_metrics=POP_DIVERSITY_METRICS,
    indiv_diversity_metrics=INDIV_DIVERSITY_METRICS,
)

### Define a MSA instance and start the analysis
MetaheuristicsSimilarityAnalyzer accepts the previously defined MetaGA under `meta_ga` and the gene space of the reference metaheuristic under `target_gene_space`.

Argument options of the `run_similarity_analysis` method:
* Specify `num_comparisons` to the desired number of comparisons to perform in the analysis. The configurations of the reference metaheuristic will be generated at random. To use optimized/tuned configurations set `generate_optimized_targets` to `True`.
    - To use specific configurations of the target metaheuristic instead, provide them under `target_solutions` argument. In this case `num_comparisons` will be ignored.
* Setting `export` to `True` produces a .pkl file which can be imported for later use.
* Setting `get_info` to `True` produces a workflow diagram displaying useful information.
* Setting `generate_dataset` to `True` generates a dataset which enables the calculation of similarity metrics for final similarity assessment.
* Setting `calculate_similarity_metrics` to `True` triggers calculation of similarity metrics.

---
**_NOTE:_** Use different `prefix` value for future experiments to prevent overwriting the existing export. It can also be left unspecified to use the current datetime value.

---

In [ ]:
msa = MetaheuristicsSimilarityAnalyzer(
    meta_ga=meta_ga,
    target_gene_space=PSO_gene_space,
    base_archive_path=base_archive_path
)

target_solutions = [
    np.array([1.95, 1.09, 0.78]),
    np.array([1.87, 1.05, 0.84]),
]

msa.run_similarity_analysis(
    num_comparisons=2,
    #target_solutions=target_solutions,
    get_info=True,
    generate_dataset=True,
    num_runs=150,
    calculate_similarity_metrics=True,
    export=True,
    prefix="analysis_example",
    pkl_filename=msa_pkl_filename,
)

### Import MSA instance

Setting `export` argument to `True` in the previous step enables importing via the static method `MetaheuristicsSimilarityAnalyzer.import_from_pkl`.

---
**_NOTE:_** Make sure to provide the correct path to the exported .pkl file.

---

In [ ]:
pkl_path = os.path.join(base_archive_path, "analysis_example_PSO-FA_Schwefel", msa_pkl_filename)
print(pkl_path)
imported_msa = MetaheuristicsSimilarityAnalyzer.import_from_pkl(pkl_path)

### Export results of the analysis to LaTeX and generate a PDF
A .tex file showing results can be created:
* Set `generate_pdf` to true to convert the file to .pdf format.

Files will be saved under the corresponding analysis directory.

In [ ]:
imported_msa.export_results_to_latex(generate_pdf=True, filename="MSA_example_results")

### Plot solution evolution for each comparison
Call `plot_solutions` to create a plot showing solutions of MetaGA trough generations:
* Provide the index of the comparison under `comparison_index` to specify for which comparison the plot will be created.
* Set `all_solutions` to `True` to plot all solutions or leave it at default value `False` to plot only the best solutions of each generation.

Plots will be saved under the corresponding comparison directory.

In [ ]:
imported_msa.plot_solutions(comparison_index=0, filename="meta_ga_all_solutions_evolution", all_solutions=True)

In [ ]:
imported_msa.plot_solutions(comparison_index=0, filename="meta_ga_best_solutions_evolution")

### Plot MetaGA fitness
Call `plot_fitness` to create a plot showing the fitness of the best solution found by MetaGA in each generation:
* Provide the index of the comparison under `comparison_index` to specify for which comparison the plot will be created.

Plot will be saved under the corresponding comparison directory.

In [ ]:
imported_msa.plot_fitness(comparison_index=0, filename="meta_ga_fitness_plot")

### Plot diversity metrics
Plots showing diversity metrics can be created:
* Call `indiv_diversity_metrics_comparison` to create a plot showing values of individual diversity metrics.
    + Provide the index of the comparison under `comparison_index` to specify for which comparison the plot will be created.
* Call `pop_diversity_metrics_comparison` to create a plot showing values of population diversity metrics.
    + Provide the index of the comparison under `comparison_index` to specify for which comparison the plot will be created.
    + Set `separate` to `True` to show metrics on separate axes for better clarity.

---
**_INFO:_** Diversity metrics are shown by run index since both of the metaheuristic algorithms populations were initialized with an rng which uses run index as the seed. This can be observed in population diversity metric plots, where plot lines of both algorithms for each metric start at the same value and then diverge based on algorithms behavior.

---

In [ ]:
imported_msa.indiv_diversity_metrics_comparison(comparison_index=0, run_index=0)

In [ ]:
imported_msa.pop_diversity_metrics_comparison(comparison_index=0, run_index=0)

In [ ]:
imported_msa.pop_diversity_metrics_comparison(comparison_index=0, run_index=0, separate=True)